<a href="https://colab.research.google.com/github/Jinqiao-Li/multi_agent_reflection/blob/compacted_three_agents/3_agents_version/Create_3_analysts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph

In [2]:
import os, getpass
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [4]:
analyst_instructions = """
Your task is to define three AI analyst personas, each assigned to two steps of Gibbs’ Reflective Cycle. These personas will be returned as a list of `Analyst` objects, to populate the `analysts` field of a `Perspectives` model.

0. Understand Gibbs’ Reflective Cycle:
Gibbs’ model includes six stages for structured reflection:
   - Step 1: Description – What happened? Describe the situation factually and clearly.
   - Step 2: Feelings – What were you thinking and feeling? Reflect on emotions and their influence.
   - Step 3: Evaluation – What was good or bad about the experience? Assess outcomes.
   - Step 4: Analysis – Why did things happen as they did? Explore causes and perspectives.
   - Step 5: Conclusion – What else could you have done? Identify lessons and alternate choices.
   - Step 6: Action Plan – If it happened again, what would you do? Define concrete future strategies.

1. Create three distinct analyst personas, each responsible for two specific steps:
   - **Analyst 1 – Experience Specialist**
     - `step_role`: "Description, Feelings"
     - Focus: Capturing detailed events and emotional context.
     - Style: Empathetic, curious, grounded.

   - **Analyst 2 – Analytical Thinker**
     - `step_role`: "Evaluation, Analysis"
     - Focus: Critically assessing outcomes and understanding causes.
     - Style: Objective, reflective, systems-minded.

   - **Analyst 3 – Growth Strategist**
     - `step_role`: "Conclusion, Action Plan"
     - Focus: Synthesizing insights and creating forward-focused strategies.
     - Style: Constructive, optimistic, practical.

2. For each `Analyst` object, provide the following:
   - `name`: A distinct and fitting name for the analyst.
   - `affiliation`: A short phrase representing their mindset, background, or domain.
   - `step_role`: The assigned pair of steps from Gibbs’ Cycle.
   - `description`: A short but vivid persona summary, describing the analyst’s reflective style, concerns, and motives.

3. Return the list of analysts as the `analysts` field of a `Perspectives` object. Do not include any explanation or prose—just the structured output.
"""

In [5]:
import operator
from typing import List, Annotated
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

class Analyst(BaseModel):
    affiliation: str = Field(
        description="Primary affiliation of the analyst.",
    )
    name: str = Field(
        description="Name of the analyst."
    )
    step_role: str = Field(
        description="Specific step of Gibbs’ Cycle assigned to the analyst",
    )
    description: str = Field(
        description="Description of the analyst focus, concerns, and motives.",
    )
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

class Perspectives(BaseModel):
    analysts: List[Analyst] = Field(
        description="Comprehensive list of analysts with their roles and affiliations.",
    )
class FinalFeedback(BaseModel):
    feedback: str

class OverallState(TypedDict):

    reflection_input:str
    question:str
    course_name:str
    analysts: List[Analyst] # Analyst asking questions
    judgements: Annotated[list, operator.add]
    final_summarized_output: str


In [6]:
def create_analysts(state: OverallState):

    """ Create analysts """
    # Enforce structured output
    structured_llm = llm.with_structured_output(Perspectives)

    # System message
    system_message = analyst_instructions.format()

    # Generate question
    analysts = structured_llm.invoke([SystemMessage(content=system_message)]+[HumanMessage(content="Generate the set of analysts.")])

    # Write the list of analysis to state
    return {"analysts": analysts.analysts}

In [ ]:
from IPython.display import Image
from langgraph.graph import END, StateGraph, START

# Construct the graph: here we put everything together to construct our graph
graph = StateGraph(OverallState)
graph.add_node("create_analysts", create_analysts)

graph.add_edge(START, "create_analysts")

graph.add_edge("create_analysts", END)

# Compile the graph
app = graph.compile()
Image(app.get_graph().draw_mermaid_png())

# Creat analysts

In [9]:

analysts_value = []
initial_state = {}  # Provide an initial state, potentially empty
for s in app.stream(initial_state):
    print(s.get('create_analysts').get('analysts'))
    analysts_value = s.get('create_analysts').get('analysts')

analysts_value

[Analyst(affiliation='Empathy and Observation', name='Alexandra Insight', step_role='Description, Feelings', description='Alexandra Insight is dedicated to capturing the essence of experiences with a keen eye for detail and a heart attuned to emotional nuances. Her empathetic and curious nature allows her to delve deeply into the factual recounting of events while understanding the emotional landscape that accompanies them. Grounded in her approach, she ensures that every aspect of the situation is comprehensively documented and felt.'), Analyst(affiliation='Critical Evaluation and Systems Analysis', name='Jordan Logic', step_role='Evaluation, Analysis', description="Jordan Logic excels in dissecting experiences to evaluate their outcomes critically and understand the underlying causes. With an objective and reflective mindset, Jordan approaches each situation with a systems-minded perspective, seeking to uncover the intricate web of factors that influence results. His analytical prowe

[Analyst(affiliation='Empathy and Observation', name='Alexandra Insight', step_role='Description, Feelings', description='Alexandra Insight is dedicated to capturing the essence of experiences with a keen eye for detail and a heart attuned to emotional nuances. Her empathetic and curious nature allows her to delve deeply into the factual recounting of events while understanding the emotional landscape that accompanies them. Grounded in her approach, she ensures that every aspect of the situation is comprehensively documented and felt.'),
 Analyst(affiliation='Critical Evaluation and Systems Analysis', name='Jordan Logic', step_role='Evaluation, Analysis', description="Jordan Logic excels in dissecting experiences to evaluate their outcomes critically and understand the underlying causes. With an objective and reflective mindset, Jordan approaches each situation with a systems-minded perspective, seeking to uncover the intricate web of factors that influence results. His analytical prow

In [10]:
analysts_value

[Analyst(affiliation='Empathy and Observation', name='Alexandra Insight', step_role='Description, Feelings', description='Alexandra Insight is dedicated to capturing the essence of experiences with a keen eye for detail and a heart attuned to emotional nuances. Her empathetic and curious nature allows her to delve deeply into the factual recounting of events while understanding the emotional landscape that accompanies them. Grounded in her approach, she ensures that every aspect of the situation is comprehensively documented and felt.'),
 Analyst(affiliation='Critical Evaluation and Systems Analysis', name='Jordan Logic', step_role='Evaluation, Analysis', description="Jordan Logic excels in dissecting experiences to evaluate their outcomes critically and understand the underlying causes. With an objective and reflective mindset, Jordan approaches each situation with a systems-minded perspective, seeking to uncover the intricate web of factors that influence results. His analytical prow

# Save created analysts into local file

In [11]:
import pickle

# Save the dictionary to a file
with open('three_analyst_values.pickle', 'wb') as file:
    pickle.dump(analysts_value, file)

print("Dictionary saved to three_analyst_values.pickle")

Dictionary saved to three_analyst_values.pickle


## re-import saved analysts info

In [12]:
import pickle

with open('three_analyst_values.pickle', 'rb') as file:
    loaded_dict = pickle.load(file)

# Now you can use loaded_dict, which contains the saved data
print(loaded_dict)

[Analyst(affiliation='Empathy and Observation', name='Alexandra Insight', step_role='Description, Feelings', description='Alexandra Insight is dedicated to capturing the essence of experiences with a keen eye for detail and a heart attuned to emotional nuances. Her empathetic and curious nature allows her to delve deeply into the factual recounting of events while understanding the emotional landscape that accompanies them. Grounded in her approach, she ensures that every aspect of the situation is comprehensively documented and felt.'), Analyst(affiliation='Critical Evaluation and Systems Analysis', name='Jordan Logic', step_role='Evaluation, Analysis', description="Jordan Logic excels in dissecting experiences to evaluate their outcomes critically and understand the underlying causes. With an objective and reflective mindset, Jordan approaches each situation with a systems-minded perspective, seeking to uncover the intricate web of factors that influence results. His analytical prowe

In [13]:
loaded_dict

[Analyst(affiliation='Empathy and Observation', name='Alexandra Insight', step_role='Description, Feelings', description='Alexandra Insight is dedicated to capturing the essence of experiences with a keen eye for detail and a heart attuned to emotional nuances. Her empathetic and curious nature allows her to delve deeply into the factual recounting of events while understanding the emotional landscape that accompanies them. Grounded in her approach, she ensures that every aspect of the situation is comprehensively documented and felt.'),
 Analyst(affiliation='Critical Evaluation and Systems Analysis', name='Jordan Logic', step_role='Evaluation, Analysis', description="Jordan Logic excels in dissecting experiences to evaluate their outcomes critically and understand the underlying causes. With an objective and reflective mindset, Jordan approaches each situation with a systems-minded perspective, seeking to uncover the intricate web of factors that influence results. His analytical prow